In [2]:
import re
from pymongo import MongoClient 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict('E:/dics/dict_keyw.txt')  #
jieba.load_userdict('E:/dics/dict_cbdic.txt')  #
#預設就是自己
client = MongoClient('10.120.28.14',27017)  #不用修改  字典檔請改位子
database = client['news']
collection =database['news']

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.480 seconds.
Prefix dict has been built succesfully.


# 執行前修改comp:報社   date:修改年份(下方共有3個find需修改)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import time

for ii in range (1,10):   # 1-9月    10月後改為10-13再執行
    a=collection.find(
        {"$and":[                   
                {"comp":{"$regex":"ebc"}},
                {"date":{"$regex":"20150{}".format(ii)}}           #計算當月文章數
                ]},{"_id":0}).count()    
    content =[]
    tStart=time.time() #計算所需時間(計時開始)
    
    for post in collection.find(
        {"$and":[                   
                {"comp":{"$regex":"ebc"}},                              #尋找一個月新聞(報社手動修改)
                {"date":{"$regex":"20150{}".format(ii)}}        #10-12月請將0去掉
                ]}): 
        summary = post['content']
        content.append('/'.join(jieba.cut(summary)))

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(content)  # titile 放文本
    weight = X.toarray()
    features = vectorizer.get_feature_names()     # 拿到所有的關鍵詞  

    top_features = []
    for n in range(0,a):  #迴圈參考上面的總文章數
        indices = np.argsort(weight[n])[::-1]  # transformer = TfidfTransformer()  #X.toarray()[5] 是第幾篇新聞的意思
        # 看TOP多少的詞
        top_n = 40
        top_features.append([features[i] for i in indices[:top_n]])    #這邊的寫法會讓關鍵字中間會有空白     

    obj=collection.find(
        {"$and":[                                                                  #尋找一個月新聞(報社手動修改)
                {"comp":{"$regex":"ebc"}},                               
                {"date":{"$regex":"20150{}".format(ii)}}
                ]})
    for i in range(0,a):  #迴圈參考上面的總文章數
        collection.update({"_id":obj[i]["_id"]},{"$set":{"tfidf":top_features[i]}}) #collection.update即可

    tEnd = time.time() #記時(時間結束)
    print '完成',len(top_features),'筆'    

    
    
    del content[:]  #清除list暫存記憶體
    del top_features[:]  #清除list暫存記憶體
    
    print ' 花了 {} s'.format(tEnd-tStart)    #將經過時間print 出來

C:\Users\BIG DATA\Anaconda2\lib\site-packages\ipykernel\__main__.py:40: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


完成 11985 筆
 花了 9632.61599994 s
完成 10255 筆
 花了 8017.80100012 s
完成 13213 筆
 花了 9225.49699998 s
完成 12322 筆
 花了 7894.35099983 s
完成 12656 筆
 花了 7970.48800015 s
完成 12649 筆
 花了 8084.96399999 s
完成 13733 筆
 花了 10666.579 s
完成 13253 筆
 花了 11118.25 s
完成 13467 筆
 花了 11885.26 s
